In [ ]:
import os

import h5py
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from src.data.tf_data import TFDataCreator
from src.data.data_augmentation import preprocess_ct
from src.models.models import Unet, BLRIUnet

%matplotlib inline

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
task = "Task04_Hippocampus"

In [ ]:
file = h5py.File(f"../data/processed/{task}/{task}_training.hdf5", "r")

In [ ]:
data_creator = TFDataCreator.get(task.split("_")[0])(file,
                                                     shuffle=True,
                                                     params_augmentation={
                                                         "rotation": False,
                                                         "random_center":
                                                         False,
                                                     })
ds = data_creator.get_tf_data(data_augmentation=True)


In [ ]:
df = pd.DataFrame()
for k in file.keys():
    shape = file[k]["image"][()].shape
    df = pd.concat([df, pd.DataFrame({
        "shape_0": [shape[0]],
        "shape_1": [shape[1]],
        "shape_2": [shape[2]],
    }, index=[k])])


In [ ]:
image = file[k]["image"][()]
label = file[k]["label"][()]
image.shape

In [ ]:
image, label = data_creator._pad_image(image, label)
image.shape

In [ ]:
yo = ds.batch(4).as_numpy_iterator()

In [ ]:
x, y = next(yo)

In [ ]:
np.std(x)

In [ ]:
y.shape

In [ ]:
s = 32
b = 3
plt.subplot(1, 4, 1)
plt.imshow(x[b, :, :, s, 0])
plt.subplot(1, 4, 2)
plt.imshow(y[b, :, :, s, 0])
plt.subplot(1, 4, 3)
plt.imshow(y[b, :, :, s, 1])
plt.colorbar()
plt.subplot(1, 4, 4)
plt.imshow(y[b, :, :, s, 2])


In [ ]:
model = Unet(output_channels=2, last_activation="softmax", n_features=[12, 24, 48, 96, 192])

In [ ]:
y_pred = model(x)

In [ ]:
s = 32
b = 0
plt.subplot(1, 3, 1)
plt.imshow(x[b, :, :, s, 0])
plt.subplot(1, 3, 2)
plt.imshow(y[b, :, :, s, 0])
plt.subplot(1, 3, 3)
plt.imshow(y_pred[b, :, :, s, 0])


In [ ]:
np.unique(y)

In [ ]:
from src.models.losses import dice_loss

In [ ]:
x.shape

In [ ]:
f = lambda y_true, y_pred: dice_loss(y_true[..., 0], y_pred[..., 0])


In [ ]:
f(y, model(x)).numpy()